In [1]:
#| default_exp 08_map-wikipedia-metadata

In [2]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [3]:
#| export
import argparse, difflib, os, pandas as pd
from tqdm.auto import tqdm

In [ ]:
#| export
from sugar.helper import *

In [5]:
#| export
def _gather_redirects(data_dir):
    files = os.listdir(data_dir)
    redirects = []
    for fn in tqdm(files):
        with open(f'{data_dir}/{fn}', 'r') as f:
            src, targ = f.read()[:-1].split('->', maxsplit=1)
            src_id = fn.split('.')[0]
        redirects.append((src_id, src, targ))
    return pd.DataFrame(redirects, columns=['source_id', 'source', 'target'])

def gather_redirects(data_dir):
    data_dir = data_dir[:-1] if data_dir[-1] == '/' else data_dir
    dirname = os.path.dirname(data_dir)
    fname = os.path.basename(data_dir)

    fname = f'{dirname}/{fname}.csv'
    if os.path.exists(fname):
        return pd.read_csv(fname)
    else:
        redirects = _gather_redirects(data_dir)
        redirects.to_csv(fname, index=False)
        return redirects
        

In [7]:
#| export
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--src_file', type=str, required=True)
    parser.add_argument('--targ_file', type=str, required=True)
    parser.add_argument('--save_file', type=str, required=True)
    parser.add_argument('--redirect_dir', type=str, required=True)
    return parser.parse_args()
    

In [8]:
#| export
if __name__ == '__main__':
    args = parse_args()
    src_ids, abstract = load_raw_txt(args.src_file, encoding='utf-8')
    targ_ids, _ = load_raw_txt(args.targ_file, encoding='latin-1')
    src_ids, norm_targ_ids = [o[11:] for o in src_ids], [o.replace('_', ' ') for o in targ_ids]

    n_common = len(set(norm_targ_ids).intersection(src_ids))
    print(f'Before intersection: {n_common}/{len(targ_ids)}.')

    redirects = gather_redirects(args.redirect_dir)
    redirects = {k:v for k,v in zip(redirects['source'], redirects['target'])}
    norm_targ_ids = [redirects[o] if o in redirects else o for o in norm_targ_ids]

    n_common = len(set(norm_targ_ids).intersection(src_ids))
    print(f'After intersection: {n_common}/{len(targ_ids)}.')

    abstracts = {k:v for k,v in zip(src_ids, abstract)}
    targ_abstract = [f'{o},{abstracts[o]}'.encode('latin-1', errors='ignore').decode('latin-1') if o in abstracts else o for o in norm_targ_ids]
    save_raw_txt(args.save_file, targ_ids, targ_abstract, encoding='latin-1')